In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
# Preprocessing the data
data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [4]:
# Encode categorical variables
labelencoder_gender = LabelEncoder()
data['Gender'] = labelencoder_gender.fit_transform(data['Gender'])

In [5]:
# One Hot Encoding
encoder_onehot_geography = OneHotEncoder()
geo_encoder = encoder_onehot_geography.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=encoder_onehot_geography.get_feature_names_out(['Geography']))

In [6]:
# combining data 
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)

In [7]:
data.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [8]:
# split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# Save the Encoders
with open('labelencoder_gender.pkl','wb') as file:
    pickle.dump(labelencoder_gender, file)

with open('OneHotEncoder_Geography', 'wb') as file:
    pickle.dump(encoder_onehot_geography, file)

# saving scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression

##

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1) # output layer
])

# compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = 'regressionlogs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [24]:
# Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100


219/219 [==============================] - 2s 3ms/step - loss: 100794.1797 - mae: 100794.1797 - val_loss: 98253.3047 - val_mae: 98253.3047
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 100377.5781 - mae: 100377.5781 - val_loss: 97405.8359 - val_mae: 97405.8359
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 98894.2344 - mae: 98894.2344 - val_loss: 95183.5938 - val_mae: 95183.5938
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 95848.1250 - mae: 95848.1250 - val_loss: 91246.4844 - val_mae: 91246.4844
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 91126.9375 - mae: 91126.9375 - val_loss: 85695.3828 - val_mae: 85695.3828
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 84902.3359 - mae: 84902.3359 - val_loss: 78931.8516 - val_mae: 78931.8516
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 77744.4766 - mae: 77744.4

In [25]:
%load_ext tensorboard/fit

In [27]:
%tensorboard --logdir regressionlogs/

Reusing TensorBoard on port 6006 (pid 8124), started 0:00:14 ago. (Use '!kill 8124' to kill it.)

In [28]:
# Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE : {test_mae}')

94/94 [==============================] - 0s 1ms/step - loss: 49776.1289 - mae: 49776.1289
Test MAE : 49776.12890625


In [29]:
model.save('regression_model.h5')

c:\projects\ANN_Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
